# "Исследование данных об инвестиции венчурных фондов в компании-стартапы."

1. Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(status) AS company_closed
FROM company
WHERE status = 'closed';

Результат:
2584

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA'
    AND category_code = 'news' 
ORDER BY funding_total DESC;

Результат:
6.22553e+08
2.5e+08
1.605e+08
1.28e+08
1.265e+08

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
      AND EXTRACT(YEAR FROM CAST (acquired_at AS date)) BETWEEN '2011' AND '2013'

Результат:
1.37762e+11

4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'

Результат:
first_name last_name twitter_username
Rebecca    Silver    SilverRebecca
Silver     Teede     SilverMatrixx
Mattias    Guilotte  Silverreven

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE last_name LIKE 'K%' AND twitter_username LIKE '%money%'

Результат:
id    first_name last_name company_id twitter_username created_at updated_at
63081 Gregory    Kim                   gmoney75        2010-07-13 2011-12-12
                                                       03:46:28   22:01:34                                       

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC

Результат:
country_code sum
USA          3.10588e+11
GBR          1.77056e+10
             1.08559e+10
CHN          1.06897e+10
CAN          9.86636e+09
IND          6.14141e+09
DEU          5.76577e+09
FRA          4.59514e+09
ISR          4.48009e+09
CHE          2.82925e+09
NLD          2.35397e+09
RUS          2.29394e+09
ESP          2.13734e+09
IRL          2.08067e+09
AUS          1.67241e+09
JPN          1.62262e+09
SWE          1.60811e+09
MYS          1.52436e+09
LUX          1.11175e+09
BRA          1.1036e+09
DNK          1.09057e+09
BEL          1.07745e+09
FIN          1.05192e+09
SGP          8.22763e+08
NOR          7.85523e+08
BMU          7.51666e+08
ITA          6.24829e+08
TWN          5.47836e+08
CZE          5.42211e+08
AUT          5.40677e+08
NZL          5.29353e+08

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
       MAX(raised_amount),
       MIN(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0 AND MIN(raised_amount) != MAX(raised_amount)

Результат:
funded_at    max         min
2012-08-22   7.5e+07     40000
2010-07-25   9e+06       3.27825e+06
2002-03-01   8.95915e+06 2.84418e+06
2010-10-11   2e+08       28000
2007-01-18   2.3e+07     5.5e+06
2007-02-27   3.6e+07     1.29e+06
2006-01-05   2.65e+07    8.9e+06
2011-10-31   2.5e+07     35000
2012-10-27   9.3e+06     500000
2007-08-16   9e+06       2.51989e+06

8. Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию "high_activity".
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию "middle_activity".
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию "low_activity".<br>
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
CASE
    WHEN invested_companies >= 100 THEN 'high_activity'
    WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
    WHEN invested_companies < 20 THEN 'low_activity'
    END
FROM fund;

Результат:
id    name             founded_at  domain      twitter_username  country_code  investment_rounds  invested_companies  milestones сreated_at          updated_at          case
13131                                                                          0                  0                   0          2013-08-19 18:46:55 2013-08-19 19:55:07 low_activity
1    Greylock Partners 1965-01-01 greylock.com greylockvc        USA           307                196                 0          2007-05-25 20:18:23 2012-12-27 00:42:24 high_activity

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY AVG(investment_rounds);

Результат:
activity        round
low_activity    2
middle_activity 51
high_activity   252

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST (founded_at AS date)) BETWEEN '2010' AND '2012'
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;

Результат:
country_code min max avg
BGR          25  35  30
CHL          29  29  29
UKR          8   10  9
LTU          5   5   5
IRL          4   5   4.5
KEN          3   3   3
LBN          3   3   3
MUS          3   3   3
JPN          1   6   2.83333
HKG          2   3   2.66667

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT OUTER JOIN education AS e ON p.id = e.person_id

Результат:
first_name last_name     instituition
John       Green         Washington University, St. Louis
John       Green         Boston University
David      Peters        Rice University
Dan        Birdwhistell  University of Cambridge
Gal        Cohen         Tel Aviv University
Chris      Treadaway     University of Texas
Chris      Treadaway     Louisiana State University
Sam        Lessin        Harvard University
Guy        Levy-Yurista  University of Pennsylvania - The Wharton School
James M.   Butler        University of Maryland

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition) AS inst
FROM education AS e
LEFT OUTER JOIN people AS p ON e.person_id = p.id
LEFT OUTER JOIN company AS c ON p.company_id = c.id
WHERE c.name IS NOT NULL
GROUP BY c.name
ORDER BY inst DESC
LIMIT 5;

Результат:
name               inst
Google             167
Yahoo!             115
Microsoft          111
Knight Foundation  74
Comcast            66

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT name
FROM company
WHERE id IN (SELECT c.id
                    FROM company AS c
                    JOIN funding_round AS fr ON c.id = fr.company_id AND (is_first_round = 1 AND is_last_round = 1))
      AND status = 'closed';
    
Результат:
name
10BestThings
11i Solutions
169 ST.
1bib
1Cast
1DayMakeover
25eight
27 Perry
2Win-Solutions
3Touch

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT id
FROM people
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE status = 'closed')
      AND company_id IN (SELECT c.id
                     FROM company AS c
                     JOIN funding_round AS fr ON c.id = fr.company_id AND (is_first_round = 1 AND is_last_round = 1));

Результат:
id
100690
10000
10053
101946
102421
102056
102997
10347
103086
1036

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id,
       e.instituition
FROM people AS p JOIN education AS e ON p.id = e.person_id
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE status = 'closed')
      AND company_id IN (SELECT c.id
                     FROM company AS c
                     JOIN funding_round AS fr ON c.id = fr.company_id AND (is_first_round = 1 AND is_last_round = 1));
    
Результат:
id    instituition
349   AKI
349   ArtEZ Hogeschool voor de Kunsten
349   Rijks Akademie
699   Imperial College
779   Harvard University
779   Stanford University
968   University of Notre Dame
972   The University of Texas at Austin
1107  CDI, Sydney
1444  Brown University

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT DISTINCT p.id,
       COUNT(e.instituition)
FROM people AS p JOIN education AS e ON p.id = e.person_id
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE status = 'closed')
      AND company_id IN (SELECT c.id
                     FROM company AS c
                     JOIN funding_round AS fr ON c.id = fr.company_id AND                              (is_first_round = 1 AND is_last_round = 1))
GROUP BY p.id;

Результат:
id    count
349   3
699   1
779   2
968   1
972   1
1107  1
1444  2
1833  1
1911  1
2008  2

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
WITH company_instituition AS (
SELECT p.id, COUNT(instituition) as count_inst
FROM education e 
       JOIN people p ON e.person_id = p.id
       JOIN company c ON c.id = p.company_id
WHERE company_id IN (SELECT id
                     FROM company 
                     WHERE id IN (SELECT company_id
                                  FROM funding_round
                                  WHERE is_first_round = 1 AND is_last_round =1)
                                  AND status = 'closed')   
GROUP BY p.id
)
SELECT SUM(count_inst)/COUNT(id) as avg_inst
FROM company_instituition

Результат:
avg_inst
1.41509

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
WITH company_instituition AS (
SELECT p.id, c.name ,COUNT(instituition) as count_inst
FROM education e 
       JOIN people p ON e.person_id = p.id
       JOIN company c ON c.id = p.company_id
WHERE c.name='Facebook'       
GROUP BY c.name , p.id
)
SELECT SUM(count_inst)/COUNT(id) as avg_inst
FROM company_instituition

Результат:
avg_inst
1.51111

19. Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.<br>
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT fu.name AS name_of_fund,
       co.name AS name_of_company,
     fr.raised_amount AS amount
FROM investment i
   JOIN fund fu ON i.fund_id=fu.id
     JOIN company co ON i.company_id=co.id
         JOIN funding_round fr on i.funding_round_id=fr.id
WHERE EXTRACT(YEAR FROM fr.funded_at) in (2012,2013)  AND  co.milestones>6

Результат:
name_of_fund        name_of_company        amount
SAP Ventures        OpenX                  2.50112e+07
Samsung Ventures    OpenX                  2.50112e+07
Index Ventures      OpenX                  2.50112e+07
Presidio Ventures   OpenX                  2.50112e+07
DAG Ventures        Gigya                  2.5e+07

20. Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.<br>
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.<br>
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
WITH company_buyer  AS (
SELECT c.name as company_buyer,
       a.price_amount as sum_buyer,
       a.id 
FROM acquisition AS a 
     JOIN company c ON c.id = a.acquiring_company_id
WHERE  a.price_amount != 0),
       company_seller AS (
       SELECT a.id,
              c.name AS company_seller,
              funding_total AS sum_invest
       FROM acquisition AS a 
            JOIN company c ON c.id = a.acquired_company_id
       WHERE funding_total != 0)
    
Результат:
company_buyer           sum_buyer            company_seller        sum_invest        part
Microsoft               8.5e+09              Skype                 7.6805e+07        111
Scout Labs              4.9e+09              Varian Semiconductor  4.8e+06           1021
                                             Equipment Associates  
Broadcom                3.7e+09              Aeluros               7.97e+06          464
Broadcom                3.7e+09              NetLogic Microsystems 1.88527e+08       20
Level 3 Communications  3e+09                Global Crossing       4.1e+07           73

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name AS category_name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS fr_month     
FROM funding_round AS fr LEFT JOIN company AS c ON fr.company_id = c.id
WHERE EXTRACT(YEAR FROM cast(fr.funded_at AS date)) BETWEEN '2010' AND '2013'
     AND c.category_code = 'social'
     AND fr.raised_amount != 0;
        
Результат:
category_name        fr_month
Klout                1
WorkSimple           3
HengZhi              1
Twitter              1
SocialGO             1
ThisNext             1
Tagged               1
LikeMe.Net           2
Busuu                10
NetBase Solutions    3

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH table_1 AS
  (SELECT EXTRACT(MONTH
                  FROM fr.funded_at) AS month_1,
          COUNT(DISTINCT f.name) AS uniq
   FROM funding_round AS fr
   LEFT JOIN investment AS i ON fr.id = i.funding_round_id
   LEFT JOIN fund AS f ON f.id = i.fund_id
   WHERE EXTRACT(YEAR
                 FROM fr.funded_at) BETWEEN 2010 AND 2013
     AND f.country_code = 'USA'
   GROUP BY month_1),
     table_2 AS
  (SELECT EXTRACT(MONTH
                  FROM a.acquired_at) AS month_2,
          COUNT(a.acquired_company_id) AS comp,
          SUM(a.price_amount) AS summa
   FROM acquisition AS a
   WHERE EXTRACT(YEAR
                 FROM a.acquired_at) BETWEEN 2010 AND 2013
   GROUP BY month_2)
SELECT table_1.month_1,
       table_1.uniq,
       table_2.comp,
       table_2.summa
FROM table_1
JOIN table_2 ON table_1.month_1 = table_2.month_2

Результат:
month_1    uniq    comp    summa
1          815     600     2.71083e+10
2          637     418     4.13903e+10
3          695     458     5.95016e+10
4          718     411     3.03837e+10
5          695     532     8.60122e+10
6          785     525     5.20883e+10
7          803     488     4.98541e+10
8          726     454     7.77093e+10
9          793     491     6.97409e+10
10         764     473     4.85567e+10

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
table_1 AS (SELECT country_code,
                   AVG(funding_total) as avg_2011
            FROM company
            WHERE EXTRACT(YEAR FROM founded_at) = 2011
            GROUP BY country_code),
table_2 AS (SELECT country_code,
                   AVG(funding_total) as avg_2012
            FROM company
            WHERE EXTRACT(YEAR FROM founded_at) = 2012
            GROUP BY country_code),
table_3 AS (SELECT country_code,
                   AVG(funding_total) as avg_2013
            FROM company
            WHERE EXTRACT(YEAR FROM founded_at) = 2013
            GROUP BY country_code) 
            
SELECT (table_1.country_code) as country,
       table_1.avg_2011,
       table_2.avg_2012,
       table_3.avg_2013
FROM table_1 LEFT JOIN table_2 ON table_1.country_code=table_2.country_code
JOIN table_3 ON table_2.country_code=table_3.country_code
ORDER BY table_1.avg_2011 DESC;

Результат:
country    avg_2011      avg_2012      avg_2013
PER        4e+06         41000         25000
USA        2.24396e+06   1.20671e+06   1.09336e+06
HKG        2.18078e+06   226227        0
PHL        1.75e+06      4218.75       2500
ARE        1.718e+06     197222        35333.3
JPN        1.66431e+06   674720        50000
AUT        1.5342e+06    147806        85773.3
BRA        1.38007e+06   240639        67944.4
DEU        1.1288e+06    1.32915e+06   66612.7
ISR        1.03076e+06   1.27121e+06   294022